# Set the environment

In [1]:
# set the envoiroment
import cobra
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import flux_variability_analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import escher
from escher import Builder
from utils import show_map
from utils.check_precursor_problem import check_precursor_problem


## load the model

In [2]:
model = cobra.io.load_json_model("../models/updated_model.json")# Import the model.
model.solver = 'glpk'
model.reactions.NGAM.bounds = (1.9,1000)
model.objective = model.reactions.Biomass
map_loc = "../data/overallmap.json"
map_loc1 = "../data/7.21 overallmap.json" #Import the map that has been drawn using escher.

## PK

### PGK and GAPDH pathway + PK + SoxABCD

In [3]:
M= model.copy()
sol = M.optimize()
l = sol.fluxes[abs(sol.fluxes)>1e-6].index
print ("Biomass flux:", sol.fluxes.loc["Biomass"])
b = show_map(sol,map_loc)
b

Biomass flux: 0.07150569777426968


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.071506, 'Biomass…

### PGK and GAPDH pathway + PK + SoxM

In [4]:
M= model.copy()
M.reactions.get_by_id("oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240").bounds = (0,0)
sol = M.optimize()
l = sol.fluxes[abs(sol.fluxes)>1e-6].index
print ("Biomass flux:", sol.fluxes.loc["Biomass"])
b = show_map(sol,map_loc1)
b

Biomass flux: 0.05094562981098347


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.050946, 'Biomass…

In [5]:
model.reactions.get_by_id('cof_sulfopterin_6.3.4.3_FORMATETHFLIG__RXN').bounds = (0,0)

### PGK and GAPDH pathway + PK + SoxM

In [6]:
M= model.copy()
M.reactions.get_by_id("oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240").bounds = (0,0)
sol = M.optimize()
l = sol.fluxes[abs(sol.fluxes)>1e-6].index
print ("Biomass flux:", sol.fluxes.loc["Biomass"])
b = show_map(sol,map_loc)
b

Biomass flux: 0.050872208992367114


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.050872, 'Biomass…

### Sox ABCD + Non-phosphorylation pathway after blocking the reactions for KDGK
    - There are two reactions for KDGK enzyme, after making the bounds to 0, the 
    non-phosphorylation pathway will be used. 
    
    - In the figure we can see that GAPDH reactions will also be used, the reason for this is the NADPH produced in the GK reaction need to be changed to NADH, and these reactions are used for recycling of NADPH. 

In [7]:
M= model.copy()
# The following two reactions are the ones for KDGK reactions
M.reactions.get_by_id('carb_entner_2.7.1.178_DEHYDDEOXGALACTKIN__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_entner_2.7.1.178_DEOXYGLUCONOKIN__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)

sol = M.optimize()
print ("Biomass flux:", sol.fluxes.loc["Biomass"])
b = show_map(sol,map_loc)
b

Biomass flux: 0.06566775263247296


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.065668, 'Biomass…

### Sox M + Non-phosphorylation pathway after blocking the reactions for KDGK
    - There are two reactions for KDGK enzyme, after making the bounds to 0, the 
    non-phosphorylation pathway will be used. 
    
    - In the figure we can see that GAPDH reactions will also be used, the reason for this is the NADPH produced in the GK reaction need to be changed to NADH, and these reactions are used for recycling of NADPH. 

In [8]:
M= model.copy()
# The following two reactions are the ones for KDGK reactions
M.reactions.get_by_id('carb_entner_2.7.1.178_DEHYDDEOXGALACTKIN__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_entner_2.7.1.178_DEOXYGLUCONOKIN__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id("oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240").bounds = (0,0)

sol = M.optimize()
print ("Biomass flux:", sol.fluxes.loc["Biomass"])
b = show_map(sol,map_loc)
b

Biomass flux: 0.042867313273416106


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.042867, 'Biomass…

### SoxABCD + spontaneous degradation pathway after blocking the reaction for GK

In [9]:
M= model.copy()
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_entner_2.7.1.165_GKI__RXN').bounds = (0,0)

sol = M.optimize()
print ("Biomass flux:", sol.fluxes.loc["Biomass"])
b = show_map(sol,map_loc)
b

Biomass flux: 0.06662926183436012


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.066629, 'Biomass…

### SoxM + spontaneous degradation pathway after blocking the reaction for GK

In [10]:
M= model.copy()
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_entner_2.7.1.165_GKI__RXN').bounds = (0,0)
M.reactions.get_by_id("oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240").bounds = (0,0)
sol = M.optimize()
l = sol.fluxes[abs(sol.fluxes)>1e-6].index
print ("Biomass flux:", sol.fluxes.loc["Biomass"])
b = show_map(sol,map_loc)
b

Biomass flux: 0.04354817645740722


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.043548, 'Biomass…

### SoxABCD + GAPN pathway after blocking GKI, spondegradation for BPG.
      - In the figure we can see that GAPDH reactions will also be used, the reason for this is the NADPH produced in the GAPN reaction need to be changed to NADH, and these reactions are used for recycling of NADPH. 

In [11]:
M= model.copy()
M.reactions.get_by_id('carb_entner_2.7.1.165_GKI__RXN').bounds = (0,0)
M.reactions.get_by_id('other_no__pathway_3.6.1.7_BS165362').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)

sol = M.optimize()
print ("Biomass flux:", sol.fluxes.loc["Biomass"])
b = show_map(sol,map_loc)
b

Biomass flux: 0.0666292618343594


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.066629, 'Biomass…

### SoxM + GAPN pathway after blocking GKI, spondegradation for BPG.
      - In the figure we can see that GAPDH reactions will also be used, the reason for this is the NADPH produced in the GAPN reaction need to be changed to NADH, and these reactions are used for recycling of NADPH. 

In [12]:
M= model.copy()
M.reactions.get_by_id('carb_entner_2.7.1.165_GKI__RXN').bounds = (0,0)
M.reactions.get_by_id('other_no__pathway_3.6.1.7_BS165362').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id("oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240").bounds = (0,0)
sol = M.optimize()
print ("Biomass flux:", sol.fluxes.loc["Biomass"])
b = show_map(sol,map_loc)
b

Biomass flux: 0.043548176457406974


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.043548, 'Biomass…

### SoxABCD+ methyxol

In [13]:
M= model.copy()
M.reactions.get_by_id('carb_entner_2.7.1.165_GKI__RXN').bounds = (0,0)
M.reactions.get_by_id('other_no__pathway_3.6.1.7_BS165362').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_1.2.1.9_1.2.1.9__RXN').bounds = (0,0)
sol = M.optimize()
print ("Biomass flux:", sol.fluxes.loc["Biomass"])
b = show_map(sol,map_loc)
b

Biomass flux: 0.05190734176769568


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.051907, 'Biomass…

### SoxM + methyxol

In [14]:
M= model.copy()
M.reactions.get_by_id('carb_entner_2.7.1.165_GKI__RXN').bounds = (0,0)
M.reactions.get_by_id('other_no__pathway_3.6.1.7_BS165362').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_1.2.1.9_1.2.1.9__RXN').bounds = (0,0)
M.reactions.get_by_id("oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240").bounds = (0,0)
sol = M.optimize()
print ("Biomass flux:", sol.fluxes.loc["Biomass"])
b = show_map(sol,map_loc)
b

Biomass flux: 0.02779722293481216


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.027797, 'Biomass…

## PEP carboxlyse + PEP synthesis:  pathway 2

### SoxABCD PGK and GAPDH pathway + SoxABCD

In [15]:
model.reactions.get_by_id ("aa_ser_gly_cys_sec_trp_1.1.1.95_PGLYCDEHYDROG__RXN").bounds = (0,0)

In [16]:
M= model.copy()
M.reactions.get_by_id("carb_enter_2.7.1.40_PEPDEPHOS__RXN").bounds = (0,0)
sol = M.optimize()
print ("Biomass flux:", sol.fluxes.loc["Biomass"])
b = show_map(sol,map_loc)
b

Biomass flux: 0.0


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### SoxM PGK and GAPDH pathway + PK + SoxM

In [17]:
M= model.copy()
M.reactions.get_by_id("oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240").bounds = (0,0)
M.reactions.get_by_id("carb_enter_2.7.1.40_PEPDEPHOS__RXN").bounds = (0,0)
sol = M.optimize()
print ("Biomass flux:", sol.fluxes.loc["Biomass"])
b = show_map(sol,map_loc)
b

Biomass flux: 7.164214123370604e-32


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### Sox ABCD + Non-phosphorylation pathway after blocking the reactions for KDGK
    - There are two reactions for KDGK enzyme, after making the bounds to 0, the 
    non-phosphorylation pathway will be used. 
    
    - In the figure we can see that GAPDH reactions will also be used, the reason for this is the NADPH produced in the GK reaction need to be changed to NADH, and these reactions are used for recycling of NADPH. 

In [18]:
M= model.copy()
# The following two reactions are the ones for KDGK reactions
M.reactions.get_by_id('carb_entner_2.7.1.178_DEHYDDEOXGALACTKIN__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_entner_2.7.1.178_DEOXYGLUCONOKIN__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id("carb_enter_2.7.1.40_PEPDEPHOS__RXN").bounds = (0,0)

sol = M.optimize()
print ("Biomass flux:", sol.fluxes.loc["Biomass"])
b = show_map(sol,map_loc)
b

Biomass flux: 0.0


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### Sox M + Non-phosphorylation pathway after blocking the reactions for KDGK
    - There are two reactions for KDGK enzyme, after making the bounds to 0, the 
    non-phosphorylation pathway will be used. 
    
    - In the figure we can see that GAPDH reactions will also be used, the reason for this is the NADPH produced in the GK reaction need to be changed to NADH, and these reactions are used for recycling of NADPH. 

In [19]:
M= model.copy()
# The following two reactions are the ones for KDGK reactions
M.reactions.get_by_id('carb_entner_2.7.1.178_DEHYDDEOXGALACTKIN__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_entner_2.7.1.178_DEOXYGLUCONOKIN__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id("carb_enter_2.7.1.40_PEPDEPHOS__RXN").bounds = (0,0)
M.reactions.get_by_id("oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240").bounds = (0,0)

sol = M.optimize()
print ("Biomass flux:", sol.fluxes.loc["Biomass"])
b = show_map(sol,map_loc)
b

Biomass flux: -4.34543890729033e-18


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': -0.0, 'Biomass__fu…

### SoxABCD + spontaneous degradation pathway after blocking the reaction for GK

In [20]:
M= model.copy()
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_entner_2.7.1.165_GKI__RXN').bounds = (0,0)
M.reactions.get_by_id("carb_enter_2.7.1.40_PEPDEPHOS__RXN").bounds = (0,0)
sol = M.optimize()
l = sol.fluxes[abs(sol.fluxes)>1e-6].index
print ("Biomass flux:", sol.fluxes.loc["Biomass"])
b = show_map(sol,map_loc)
b

Biomass flux: 3.675684133306823e-17


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### SoxM + spontaneous degradation pathway after blocking the reaction for GK

In [21]:
M= model.copy()
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_entner_2.7.1.165_GKI__RXN').bounds = (0,0)
M.reactions.get_by_id("carb_enter_2.7.1.40_PEPDEPHOS__RXN").bounds = (0,0)
M.reactions.get_by_id("oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240").bounds = (0,0)
sol = M.optimize()

print ("Biomass flux:", sol.fluxes.loc["Biomass"])
b = show_map(sol,map_loc)
b

Biomass flux: 0.0


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### SoxABCD + GAPN pathway after blocking GKI, spondegradation for BPG.
      - In the figure we can see that GAPDH reactions will also be used, the reason for this is the NADPH produced in the GAPN reaction need to be changed to NADH, and these reactions are used for recycling of NADPH. 

In [22]:
M= model.copy()
M.reactions.get_by_id('carb_entner_2.7.1.165_GKI__RXN').bounds = (0,0)
M.reactions.get_by_id('other_no__pathway_3.6.1.7_BS165362').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id("carb_enter_2.7.1.40_PEPDEPHOS__RXN").bounds = (0,0)
sol = M.optimize()
print ("Biomass flux:", sol.fluxes.loc["Biomass"])
b = show_map(sol,map_loc)
b

Biomass flux: -6.810881666829718e-18


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': -0.0, 'Biomass__fu…

### SoxM + GAPN pathway after blocking GKI, spondegradation for BPG.
      - In the figure we can see that GAPDH reactions will also be used, the reason for this is the NADPH produced in the GAPN reaction need to be changed to NADH, and these reactions are used for recycling of NADPH. 

In [23]:
M= model.copy()
M.reactions.get_by_id('carb_entner_2.7.1.165_GKI__RXN').bounds = (0,0)
M.reactions.get_by_id('other_no__pathway_3.6.1.7_BS165362').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id("carb_enter_2.7.1.40_PEPDEPHOS__RXN").bounds = (0,0)
M.reactions.get_by_id("oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240").bounds = (0,0)
sol = M.optimize()
print ("Biomass flux:", sol.fluxes.loc["Biomass"])
b = show_map(sol,map_loc)
b

Biomass flux: -1.6545925916188108e-17


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': -0.0, 'Biomass__fu…

## blocking PEP carboxlyse + PEP synthesis:  pathway 3

### SoxABCD PGK and GAPDH pathway + SoxABCD

In [24]:
reaction = Reaction('aa_ser_gly_thr_4.3.1.19_L_SERINE_AMMONIA_LYASE_RXN')
reaction.name = 'aa_ser_gly_thr_4.3.1.19_L_SERINE_AMMONIA_LYASE_RXN'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default
model.add_reactions([reaction])
reaction.add_metabolites({"L__serine":-1.0,
                         "pyruvate":1.0,
                         "ammonia": 1.0})
model.reactions.get_by_id('aa_met_salvage_4.2.1.22_CYSTATHIONINE__BETA__SYNTHASE__RXN').bounds = (0,0)# these two reactions do not exist in this organism

In [25]:
model.reactions.get_by_id("carb_anap_4.1.1.31_PEPCARBOX__RXN").bounds = (0,0)
model.reactions.get_by_id("aa_ser_gly_cys_sec_trp_1.1.1.95_PGLYCDEHYDROG__RXN").bounds = (0,10)

In [26]:
M= model.copy()
M.reactions.get_by_id("carb_enter_2.7.1.40_PEPDEPHOS__RXN").bounds = (0,0)
sol = M.optimize()
print ("Biomass flux:", sol.fluxes.loc["Biomass"])
b = show_map(sol,map_loc)
b

Biomass flux: 0.06750586356686311


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.067506, 'Biomass…

### SoxM PGK and GAPDH pathway + PK + SoxM

In [27]:
M= model.copy()
M.reactions.get_by_id("oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240").bounds = (0,0)
M.reactions.get_by_id("carb_enter_2.7.1.40_PEPDEPHOS__RXN").bounds = (0,0)
sol = M.optimize()
print ("Biomass flux:", sol.fluxes.loc["Biomass"])
b = show_map(sol,map_loc)
b

Biomass flux: 0.044367746303049814


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.044368, 'Biomass…

### Sox ABCD + Non-phosphorylation pathway after blocking the reactions for KDGK
    - There are two reactions for KDGK enzyme, after making the bounds to 0, the 
    non-phosphorylation pathway will be used. 
    
    - In the figure we can see that GAPDH reactions will also be used, the reason for this is the NADPH produced in the GK reaction need to be changed to NADH, and these reactions are used for recycling of NADPH. 

In [28]:
M= model.copy()
# The following two reactions are the ones for KDGK reactions
M.reactions.get_by_id('carb_entner_2.7.1.178_DEHYDDEOXGALACTKIN__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_entner_2.7.1.178_DEOXYGLUCONOKIN__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id("carb_enter_2.7.1.40_PEPDEPHOS__RXN").bounds = (0,0)

sol = M.optimize()
print ("Biomass flux:", sol.fluxes.loc["Biomass"])
b = show_map(sol,map_loc)
b

Biomass flux: 0.06155803751483595


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.061558, 'Biomass…

### Sox M + Non-phosphorylation pathway after blocking the reactions for KDGK
    - There are two reactions for KDGK enzyme, after making the bounds to 0, the 
    non-phosphorylation pathway will be used. 
    
    - In the figure we can see that GAPDH reactions will also be used, the reason for this is the NADPH produced in the GK reaction need to be changed to NADH, and these reactions are used for recycling of NADPH. 

In [29]:
M= model.copy()
# The following two reactions are the ones for KDGK reactions
M.reactions.get_by_id('carb_entner_2.7.1.178_DEHYDDEOXGALACTKIN__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_entner_2.7.1.178_DEOXYGLUCONOKIN__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id("oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240").bounds = (0,0)
M.reactions.get_by_id("carb_enter_2.7.1.40_PEPDEPHOS__RXN").bounds = (0,0)

sol = M.optimize()
print ("Biomass flux:", sol.fluxes.loc["Biomass"])
b = show_map(sol,map_loc)
b

Biomass flux: 0.03596117859586143


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.035961, 'Biomass…

### SoxABCD + GAPN pathway after blocking the reaction for GK

In [30]:
M= model.copy()
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_entner_2.7.1.165_GKI__RXN').bounds = (0,0)
M.reactions.get_by_id("carb_enter_2.7.1.40_PEPDEPHOS__RXN").bounds = (0,0)
sol = M.optimize()
print ("Biomass flux:", sol.fluxes.loc["Biomass"])
b = show_map(sol,map_loc)
b

Biomass flux: 0.062487030744228726


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.062487, 'Biomass…

### SoxM + GAPN pathway after blocking the reaction for GK

In [31]:
M= model.copy()
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_entner_2.7.1.165_GKI__RXN').bounds = (0,0)
M.reactions.get_by_id("oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240").bounds = (0,0)
M.reactions.get_by_id("carb_enter_2.7.1.40_PEPDEPHOS__RXN").bounds = (0,0)

sol = M.optimize()
l = sol.fluxes[abs(sol.fluxes)>1e-6].index
print ("Biomass flux:", sol.fluxes.loc["Biomass"])
b = show_map(sol,map_loc)
b

Biomass flux: 0.03655761159419125


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.036558, 'Biomass…

### SoxABCD + spontaneous degradation pathway after blocking GKI, spondegradation for BPG.
      - In the figure we can see that GAPDH reactions will also be used, the reason for this is the NADPH produced in the GAPN reaction need to be changed to NADH, and these reactions are used for recycling of NADPH. 

In [32]:
M= model.copy()
M.reactions.get_by_id('carb_entner_2.7.1.165_GKI__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_1.2.1.9_1.2.1.9__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id("carb_enter_2.7.1.40_PEPDEPHOS__RXN").bounds = (0,0)
sol = M.optimize()
print ("Biomass flux:", sol.fluxes.loc["Biomass"])
b = show_map(sol,map_loc)
b

Biomass flux: 0.06248703074422876


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.062487, 'Biomass…

### SoxM + spontaneous degradation pathway after blocking GKI, spondegradation for BPG.
      - In the figure we can see that GAPDH reactions will also be used, the reason for this is the NADPH produced in the GAPN reaction need to be changed to NADH, and these reactions are used for recycling of NADPH. 

In [33]:
M= model.copy()
M.reactions.get_by_id('carb_entner_2.7.1.165_GKI__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_1.2.1.9_1.2.1.9__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id("oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240").bounds = (0,0)
M.reactions.get_by_id("carb_enter_2.7.1.40_PEPDEPHOS__RXN").bounds = (0,0)

sol = M.optimize()
print ("Biomass flux:", sol.fluxes.loc["Biomass"])
b = show_map(sol,map_loc)
b

Biomass flux: 0.03655761159419117


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.036558, 'Biomass…